## Welcome to your notebook.


#### Run this cell to connect to your GIS and get started:

In [1]:
from arcgis.gis import GIS
gis = GIS("home")

/opt/conda/lib/python3.11/site-packages/arcgis/gis/__init__.py:731: UserWarning: You are logged on as zarub006_UMN with an administrator role, proceed with caution.
  warnings.warn(


#### Now you are ready to start!

In [2]:
# Let's import some other libraries
import pandas as pd
import requests
from arcgis.features import GeoAccessor, GeoSeriesAccessor

In [3]:
# Let's make a request
response = requests.get("https://restcountries.com/v3.1/all?fields=name,latlng,flag")

json_output = response.json()

In [4]:
# Now we can load that into a DataFrame
df = pd.DataFrame(json_output)

df.head()

,name,latlng,flag
0,"{'common': 'South Georgia', 'official': 'South...","[-54.5, -37.0]",🇬🇸
1,"{'common': 'Grenada', 'official': 'Grenada', '...","[12.11666666, -61.66666666]",🇬🇩
2,"{'common': 'Switzerland', 'official': 'Swiss C...","[47.0, 8.0]",🇨🇭
3,"{'common': 'Sierra Leone', 'official': 'Republ...","[8.5, -11.5]",🇸🇱
4,"{'common': 'Hungary', 'official': 'Hungary', '...","[47.0, 20.0]",🇭🇺


In [5]:
# Hmm, there is nested JSON that we need to deal with...
df = pd.json_normalize(json_output)

df.head()

,latlng,flag,name.common,name.official,name.nativeName.eng.official,name.nativeName.eng.common,name.nativeName.fra.official,name.nativeName.fra.common,name.nativeName.gsw.official,name.nativeName.gsw.common,...,name.nativeName.zib.official,name.nativeName.zib.common,name.nativeName.mon.official,name.nativeName.mon.common,name.nativeName.heb.official,name.nativeName.heb.common,name.nativeName.mri.official,name.nativeName.mri.common,name.nativeName.nzs.official,name.nativeName.nzs.common
0,"[-54.5, -37.0]",🇬🇸,South Georgia,South Georgia and the South Sandwich Islands,South Georgia and the South Sandwich Islands,South Georgia,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"[12.11666666, -61.66666666]",🇬🇩,Grenada,Grenada,Grenada,Grenada,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"[47.0, 8.0]",🇨🇭,Switzerland,Swiss Confederation,NaN,NaN,Confédération suisse,Suisse,Schweizerische Eidgenossenschaft,Schweiz,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"[8.5, -11.5]",🇸🇱,Sierra Leone,Republic of Sierra Leone,Republic of Sierra Leone,Sierra Leone,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"[47.0, 20.0]",🇭🇺,Hungary,Hungary,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
# Let's get rid of some columns that we don't need
df = df.drop([column for column in df.columns if column not in ["latlng", "flag", "name.common"]], axis=1)

# And now let's split apart lat & lon
df["LAT"] = df["latlng"].apply(lambda x: x[0])
df["LON"] = df["latlng"].apply(lambda x: x[1])

# And drop latlong
df = df.drop("latlng", axis=1)

# And rename the columns
df = df.rename({"flag": "FLAG", "name.common": "COUNTRY"}, axis=1)

df.head()

,FLAG,COUNTRY,LAT,LON
0,🇬🇸,South Georgia,-54.500000,-37.000000
1,🇬🇩,Grenada,12.116667,-61.666667
2,🇨🇭,Switzerland,47.000000,8.000000
3,🇸🇱,Sierra Leone,8.500000,-11.500000
4,🇭🇺,Hungary,47.000000,20.000000


In [7]:
# Our data is ready, but it isn't spatial yet
sedf = pd.DataFrame.spatial.from_xy(df=df, x_column="LON", y_column="LAT", sr=4326)

sedf.head()

,FLAG,COUNTRY,LAT,LON,SHAPE
0,🇬🇸,South Georgia,-54.500000,-37.000000,"{""spatialReference"": {""wkid"": 4326}, ""x"": -37...."
1,🇬🇩,Grenada,12.116667,-61.666667,"{""spatialReference"": {""wkid"": 4326}, ""x"": -61...."
2,🇨🇭,Switzerland,47.000000,8.000000,"{""spatialReference"": {""wkid"": 4326}, ""x"": 8.0,..."
3,🇸🇱,Sierra Leone,8.500000,-11.500000,"{""spatialReference"": {""wkid"": 4326}, ""x"": -11...."
4,🇭🇺,Hungary,47.000000,20.000000,"{""spatialReference"": {""wkid"": 4326}, ""x"": 20.0..."


In [8]:
# Now let's make a basic map
basic_map = gis.map()

sedf.spatial.plot(
    map_widget=basic_map,
    renderer_type="s",
    symbol_type="simple",
    symbol_style="d",
    marker_size=10,
)

# Here we are showing it below
basic_map

MapView(layout=Layout(height='400px', width='100%'))

In [9]:
# Let's learn more about the method to see if we can add popups
help(sedf.spatial.plot)

Help on method plot in module arcgis.features.geo._accessor:

plot(map_widget=None, **kwargs) method of arcgis.features.geo._accessor.GeoAccessor instance
    The ``plot`` draws the data on a web map. The user can describe in simple terms how to
    renderer spatial data using symbol.
    
    .. note::
        To make the process simpler, a palette
        for which colors are drawn from can be used instead of explicit colors.
    
    
    ======================  =========================================================
    **Explicit Argument**   **Description**
    ----------------------  ---------------------------------------------------------
    map_widget              optional ``WebMap`` object. This is the map to display
                            the data on.
    ----------------------  ---------------------------------------------------------
    palette                 optional string/dict. Color mapping. Can also be listed
                            as 'colors' or 'cmap

In [10]:
# It doesn't seem like we can customize the popup using the plot method on the SEDF, so let's try drawing each feature instead
the_map = gis.map()

symbol_config = {"type": "esriSMS", "style": "esriSMSDiamond", "color": [0,216,255,255], "size": 10}

for _, country in sedf.iterrows():
    popup_config = {"title" : country["COUNTRY"], "content": country["FLAG"]}
    the_map.draw(country["SHAPE"], popup=popup_config, symbol=symbol_config)

# Here we are showing it below
the_map

MapView(layout=Layout(height='400px', width='100%'))